<a href="https://colab.research.google.com/github/PieroPastor/sparse-multiplication/blob/main/sparse_multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! sudo apt install nasm

In [2]:
import os
import math
import time
import ctypes
import numpy as np
import scipy.sparse as sp
from itertools import repeat
from multiprocessing import Pool

In [3]:
def compressed_sparse_row_traditional(x):
  compressed = []
  row_pointer = []
  flag = True
  for i in range(len(x)):
    for j in range(len(x[i])):
      if x[i][j] != 0:
        if flag:
          row_pointer.insert(len(row_pointer), len(compressed))
          flag = False
        compressed.insert(len(compressed), [i, j, x[i][j]])
    flag = True
  compressed = np.array(compressed)
  return np.array(row_pointer).astype(np.float32), np.array(compressed[:, 0]).astype(np.float32), np.array(compressed[:, 1]).astype(np.float32), np.array(compressed[:, 2]).astype(np.float32)

In [4]:
def compressed_sparse_row(x):
  compressed = []
  cant_per_row = []
  for i in range(len(x)):
    counter = 0
    for j in range(len(x[i])):
      if x[i][j] != 0:
        compressed.insert(len(compressed), [i, j, x[i][j]])
        counter += 1
    cant_per_row.insert(len(cant_per_row), counter)
  compressed = np.array(compressed)
  return len(x), len(x[0]), np.array(cant_per_row).astype(np.int32), np.array(compressed[:, 0]).astype(np.float32), np.array(compressed[:, 1]).astype(np.float32), np.array(compressed[:, 2]).astype(np.float32)

In [5]:
def decompressed_sparse_row(row, col, val, cant_rows, cant_cols):
  sparse_new = np.zeros((cant_rows, cant_cols)).astype(np.float32)
  counter = 0
  for i in range(int(max(row))+1):
    for j in range(int(max(col))+1):
      if counter < len(row) and i == row[counter] and j == col[counter]:
        sparse_new[i][j] = val[counter]
        counter += 1
  return sparse_new

In [6]:
def matriz_vector(cant_per_row, col, val, vector):
  sol = np.zeros((int(max(row))+1), ).astype(np.float32)
  sumador = 0
  for i in range(int(max(row))+1):
    for j in range(cant_per_row[i]):
      sol[i] += val[j + int(sumador)] * vector[int(col[j + int(sumador)])]
    sumador += cant_per_row[i]
  return sol

In [36]:
def matriz_vector_traditional(row, col, val, vector):
  sol = np.zeros(len(row)).astype(np.float32)
  for i in range(len(row)-1):
    for j in range(int(row[i]), int(row[i+1])): #Se usa como tope el siguiente puntero
      sol[i] += val[j] * vector[int(col[j])]
  for i in range(int(row[len(row)-1]), len(col)): #Se itera especialmente para el último puntero
    sol[len(row)-1] += val[i] * vector[int(col[i])]
  return sol

In [7]:
%%file simd_section.asm
section .text
    global simd_section
;rdi <- *col
;rsi <- *val
;rdx <- *vector
;rcx <- size
simd_section:
    xorps xmm0, xmm0 ;El sumador
    mov r9, 0 ;Será mi índice
    cmp r9, rcx
    jl bucle_sumador
ret

bucle_sumador:
    xorps xmm1, xmm1 ;Guarda los valores de la matriz
    xorps xmm2, xmm2 ;Guarda los valores del vector
    mov r10, rcx
    sub r10, r9
    cmp r10, 4
    jl enviar_no_simd
    movups xmm1, [rsi + r9*4]
    mov r10, rcx
    mov rcx, 4
    jmp guardar_vector

enviar_no_simd:
    movss xmm3, [rsi + r9*4]
    movss xmm1, xmm3
    cvtss2si r12, [rdi + r9*4]
    movss xmm3, [rdx + r12*4]
    movss xmm2, xmm3
    shufps xmm1, xmm1, 00111001b
    shufps xmm2, xmm2, 00111001b
    inc r9
    cmp r9, rcx
    jl enviar_no_simd
    jmp fin_bucle

guardar_vector:
    cvtss2si r12, [rdi + r9*4]
    movss xmm3, [rdx + r12*4]
    movss xmm2, xmm3
    shufps xmm2, xmm2, 00111001b
    inc r9
    ;dec r10
    ;cmp r10, 0
    ;jg guardar_vector
    loop guardar_vector
    mov rcx, r10 ;Regresa rcx a la normalidad
    jmp fin_bucle

fin_bucle:
    mulps xmm1, xmm2
    haddps xmm1, xmm1
    haddps xmm1, xmm1
    addss xmm0, xmm1
    cmp r9, rcx ;Ya se le sumaron 4, o lo que restaba menor a 4 en sus bucles respectivos
    jl bucle_sumador
    ret

Writing simd_section.asm


In [8]:
! nasm -f elf64 simd_section.asm -o simd_section.o

In [9]:
%%file multithreading_section.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

extern float simd_section(float *col, float *val, float *vector, int size);

typedef struct th_info{
    pthread_t thread;
    int size;
    int id;
    float *col;
    float *val;
    float *vector;
    float suma;
} th_info;

void *pth_func(void *args){
    th_info *th_info_args = (th_info*) args;
    th_info_args->suma = simd_section(th_info_args->col, th_info_args->val, th_info_args->vector, th_info_args->size);
    return NULL;
}

float multithreading_vector(int n, float *col, float *val, float *vector, int sumador, int ths){
    float suma=0;
    int size=n/ths;
    if(n > ths)size = n/ths;
    else if(n > 0){
        size = 1;
        ths = n;
    }
    th_info threads[ths];

    for(int i=0; i < ths; i++){
        if(i == ths-1) threads[i].size = size + (n-(ths*size)); //Da el residuo como extra
        else threads[i].size = size;
        threads[i].id = i;
        threads[i].col = col + i*size + sumador;
        threads[i].val = val + i*size + sumador;
        threads[i].vector = vector;
        threads[i].suma = 0;
    }

    pth_func((void *)&threads[0]);

    if(ths > 1){
        for(size_t i=1; i < ths; i++) pthread_create(&threads[i].thread, NULL, pth_func, (void*)&threads[i]);
        for(size_t i=1; i < ths; i++) pthread_join(threads[i].thread, NULL);
    }

    for(int i=0; i < ths; i++) suma += threads[i].suma;

    return suma;
}

void matriz_vector(int *repeats, int len_rep, float *col, float *val, float *vector, float *sol, int ths){
    int sumador=0;
    for(int i=0; i < len_rep; i++){
        sol[i] = multithreading_vector(repeats[i], col, val, vector, sumador, ths);
        sumador += repeats[i];
    }
}

Writing multithreading_section.c


In [10]:
! gcc -shared -fPIC simd_section.o multithreading_section.c -o pmultithreading_c_simd.so -pthread

In [11]:
def pmultithreading_c_simd():
  lib = ctypes.CDLL('./pmultithreading_c_simd.so')
  lib.matriz_vector.argtypes = [np.ctypeslib.ndpointer(dtype = np.int32),
                                ctypes.c_int,
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                ctypes.c_int]
  return lib.matriz_vector

In [12]:
%%file matriz_vector_traditional_simd.asm
section .text
    global matriz_vector_traditional_simd
;rdi <- row
;rsi <- col
;rdx <- val
;rcx <- vector
;r8 <- sol
;r9 <- n_ptrs, n
;Se tiene un algoritmo que mezcla SIMD y al mismo tiempo operaciones normales para ciertas operaciones, ya que, no se puede predecir cuantos valores
;diferente de cero habrá en cada columna de la matriz.
matriz_vector_traditional_simd:
    movss xmm1, [r9 + 4] ;Guarda n en xmm1 ya que se mandó como doubles y se debe de convertir
    movss xmm0, [r9] ;Guarda n_ptrs
    cvtss2si r9, xmm0 ;Los convierte a enteros
    cvtss2si r10, xmm1
    dec r9 ;Le resta uno porque el bucle es hasta n_ptrs-1
    mov r12, 0 ;Valor de i
    cmp r12, r9
    jl bucle1
ret

bucle1:
    xorps xmm0, xmm0 ;Se limpian todos los registros para evitar fallos en las operaciones
    xorps xmm1, xmm1
    xorps xmm2, xmm2
    xorps xmm3, xmm3
    xorps xmm4, xmm4
    movss xmm15, [rdi + r12*4] ;Pasa la información del puntero a fila y la pasa a entero
    cvtss2si r13, xmm15
    movss xmm15, [rdi + r12*4 + 4] ;Pasa el siguiente puntero a fila como tope y lo pasa a entero
    cvtss2si r14, xmm15
    cmp r13, r14
    push r12
    jl bucle2
    jmp fin_bucle2

bucle2:
    push r13 ;Guarda el contador r13, ya que, se necesitan más registros
    mov r12, r14
    sub r12, r13
    mov rax, r12
    cmp r12, 4 ;Para ver si faltan menos de 4 manda especial, es decir, usará los registros xmmx como cola y almacenará la información ahí hasta donde haya
    jl enviar_especial
    mov r12, 4
    movups xmm1, [rdx + r13*4] ;Manda cuatro valores con una sola operacion
    movups xmm2, [rsi + r13*4]
    mov r13, r12
    jmp guardar_datos_vector

enviar_especial:
    xorps xmm4, xmm4 ;Lo almacena de esa forma para también poder operar con mulps, ya que, como los sobrantes es cero no hay problema y aún se ahorra tiempo
    movss xmm4, [rdx + r13*4] ;Se necesita el auxiliar porque el movss cuando se trabaja con un puntero limpia todo el registro, por lo que la cola se borra
    movss xmm1, xmm4
    movss xmm4, [rsi + r13*4]
    movss xmm2, xmm4
    shufps xmm1, xmm1, 00111001b ;Envía el valor guardado al final y el siguiente para guardarlo ahí (una cola)
    shufps xmm2, xmm2, 00111001b
    dec r12
    inc r13
    cmp r12, 0 ;Hasta que se ingrese la cantidad de data que haya
    jg enviar_especial
    mov rbx, 4 ;Para mantener todo alineado y operar correctamente se deben de completar las vueltas a la cola por lo que la diferencia con 4 es la cantidad de vueltas que faltan.
    sub rbx, rax
    jmp organizar_especial

organizar_especial:
    shufps xmm1, xmm1, 00111001b ;Realiza solo vueltas para alinear los datos
    shufps xmm2, xmm2, 00111001b
    dec rbx
    cmp rbx, 0
    jg organizar_especial
    mov r13, rax
    mov r12, rax
    jmp guardar_datos_vector

guardar_datos_vector:
    cvtss2si r15, xmm2 ;Realiza la misma lógica para guardar todos los datos necesarios del vector en un registro y así operarlo junto aprovechando el SIMD
    movss xmm4, [rcx + r15*4]
    movss xmm3, xmm4
    shufps xmm2, xmm2, 00111001b ;Pasa a la siguiente columna
    shufps xmm3, xmm3, 00111001b ;Para guardar el siguiente dato de vector
    dec r12
    cmp r12, 0
    jg guardar_datos_vector
    mov r12, 4
    sub r12, r13
    cmp r13, 4 ;En caso haya habido menos de 4 valores también se tendrán que alinear los datos
    jl organizar_datos
    jmp fin_bucle2

organizar_datos:
    shufps xmm3, xmm3, 00111001b
    shufps xmm2, xmm2, 00111001b
    dec r12
    cmp r12, 0
    jg organizar_datos
    jmp fin_bucle2

fin_bucle2:
    pop r13 ;Regresa r13 a la normalidad
    mulps xmm3, xmm1 ;Multiplica los datos del vector y la matriz y los suma entre ellos
    haddps xmm3, xmm3
    haddps xmm3, xmm3
    addss xmm0, xmm3 ;Se agrega al sumador
    add r13, 4 ;Se suma cuatro porque se avanzo esa cantidad si o si
    xorps xmm1, xmm1 ;Se limpian los registros
    xorps xmm2, xmm2
    xorps xmm3, xmm3
    cmp r13, r14
    jl bucle2
    pop r12 ;Se retira r12 para usarlo y guardar en soluciones el xmm0
    movss [r8 + r12*4], xmm0
    inc r12
    cmp r10, -1 ;Se tiene esta bandera especial para poder usar el código desde otro punto fuera del bucle principal
    je fin_algoritmo
    cmp r12, r9
    jl bucle1
    jmp segunda_parte

segunda_parte:
    movss xmm15, [rdi + r9*4] ;Como falta el valor para n_ptrs porque se evitó, se setean los registros y se salta al bucle2 como si fuera una función
    xorps xmm0, xmm0
    cvtss2si r13, xmm15 ;Pasa el [n_ptrs-1]
    mov r14, r10 ;El final es la cantidad de columnas
    mov r12, r9
    push r12 ;Guarda r12 más que nada por el futuro popeo y que así el PC se mantenga intacto y pueda retornar
    mov r10, -1 ;Servirá como una bandera
    xorps xmm0, xmm0
    xorps xmm1, xmm1
    xorps xmm2, xmm2
    xorps xmm3, xmm3
    xorps xmm4, xmm4
    jmp bucle2

fin_algoritmo:
    ret

Writing matriz_vector_traditional_simd.asm


In [13]:
! nasm -f elf64 matriz_vector_traditional_simd.asm -o matriz_vector_traditional_simd.o

In [14]:
%%file matriz_vector.c
extern void matriz_vector_traditional_simd(float *row, float *col, float *val, float *vector, float *sol, float *tam);
void matriz_vector(float *row, float *col, float *val, float *vector, float *sol, int *repeats, int n, int cant_data){
    int sumador=0;
    for(int i=0; i < n; i++){
        for(int j=0; j < repeats[i]; j++){
            sol[i] += val[j + sumador] * vector[(int)col[j + sumador]];
        }
        sumador += repeats[i];
    }
    return;
}
void matriz_vector_traditional(float *row, float *col, float *val, float *vector, float *sol, int n_ptrs, int n){
    for(int i=0; i < n_ptrs-1; i++){
        for(int j=(int)row[i]; j < (int)row[i+1]; j++) sol[i] += val[j] * vector[(int)col[j]];
    }
    for(int i=(int)row[n_ptrs-1]; i < n; i++) sol[n_ptrs-1] += val[i] * vector[(int)col[i]];
    return;
}

Writing matriz_vector.c


In [15]:
! gcc -shared -fPIC matriz_vector_traditional_simd.o matriz_vector.c -o matriz_vector.so

In [16]:
def matriz_vector_c(repeats, row, col, val, vector):
  sol = np.zeros((int(max(row))+1),).astype(np.float32)
  lib = ctypes.CDLL('./matriz_vector.so')
  lib.matriz_vector.argtypes = [np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.int32),
                                ctypes.c_int, ctypes.c_int]
  lib.matriz_vector(row, col, val, vector, sol, repeats, (int(max(row))+1), len(row))
  return sol

In [17]:
def matriz_vector_traditional_c(row, col, val, vector):
  sol = np.zeros(len(row),).astype(np.float32)
  lib = ctypes.CDLL('./matriz_vector.so')
  lib.matriz_vector_traditional.argtypes = [np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                np.ctypeslib.ndpointer(dtype = np.float32),
                                ctypes.c_int, ctypes.c_int]
  lib.matriz_vector_traditional(row, col, val, vector, sol, len(row), len(col))
  return sol

In [18]:
def matriz_vector_traditional_simd(row, col, val, vector):
  sol = np.zeros(len(row),).astype(np.float32)
  lib = ctypes.CDLL('./matriz_vector.so')
  lib.matriz_vector_traditional_simd.argtypes = [np.ctypeslib.ndpointer(dtype = np.float32),
                                     np.ctypeslib.ndpointer(dtype = np.float32),
                                     np.ctypeslib.ndpointer(dtype = np.float32),
                                     np.ctypeslib.ndpointer(dtype = np.float32),
                                     np.ctypeslib.ndpointer(dtype = np.float32),
                                     np.ctypeslib.ndpointer(dtype = np.float32)]
  lib.matriz_vector_traditional_simd(row, col, val, vector, sol, np.array([np.int32(len(row)), np.int32(len(col))]).astype(np.float32))
  return sol

In [19]:
def matriz_vector_multiprocessing_sub(repeats, len_rep, col, val, vector):
  sol = np.zeros(len_rep).astype(np.float32) #len_rep por la cantidad de filas que habrá
  matriz_vector_multithreading = pmultithreading_c_simd()
  matriz_vector_multithreading(repeats, len_rep, col, val, vector, sol, os.cpu_count())
  return sol

#Se debe de optimizar mandando el cant_repeats como argumento, en lugar de calcularlo aquí dentro, es decir se debe hallar al descomponer la matriz
def matriz_vector_multiprocessing(cant_repeats, row, col, val, vector):
  #cant_repeats = [list(row).count(i) for i in range(int(max(row))+1)] #Crea un arreglo con la cantidad de datos diferentes a cero por fila
  cpus = os.cpu_count()
  chunk = 1
  if cpus > max(row)+1: cpus = max(row)+1 #Si hay más procesadores que procesos necesarios, entonces se recorta
  else: chunk = math.ceil((max(row)+1)/cpus) #Si hay menos, se divide en tantos procesos como procesadores y un chunk es un grupo de datos
  cpus = int(cpus)
  p = Pool(processes=cpus) #Inicializa el pool para multiprocessing
  args = zip((cant_repeats[i*chunk:(i+1)*chunk] for i in range(cpus)), #Manda las cantidades de datos diferentes a cero en las respectivas filas
             (len(cant_repeats[i*chunk:(i+1)*chunk]) for i in range(cpus)), #Manda el tamaño real de filas para evitar desbordamiento por el trabajo con chunks y saber como moverse
             (col[sum(cant_repeats[0:i*chunk]):sum(cant_repeats[0:i*chunk])+sum(cant_repeats[i*chunk:(i+1)*chunk])] for i in range(0, cpus)), #Manda el grupo de columnas respectivas dentro de las filas, ya que está ordenado por las filas y columnas
             (val[sum(cant_repeats[0:i*chunk]):sum(cant_repeats[0:i*chunk])+sum(cant_repeats[i*chunk:(i+1)*chunk])] for i in range(0, cpus)), #Misma lógica que para las columnas
             repeat(vector)) #Envía el vector igual siempre porque se necesita para multiplicar
  resultado = p.starmap(matriz_vector_multiprocessing_sub, args)
  return np.concatenate(resultado)

In [20]:
def matriz_vector_multiprocessing_sub_p(repeats, len_rep, col, val, vector):
  sol = np.zeros(len_rep).astype(np.float32) #fin-ini por la cantidad de filas que habrá
  sumador = 0
  for i in range(len_rep): #Repite tantas filas hay por proceso
    for j in range(repeats[i]): #Itera cuantos datos diferentes a cero haya en la fila ahorrando iteraciones innecesarias
      sol[i] += val[j + int(sumador)] * vector[int(col[j + int(sumador)])] #Se salta los datos de filas anteriores con la función sum()
    sumador += repeats[i]
  return sol

#Se debe de optimizar mandando el cant_repeats como argumento, en lugar de calcularlo aquí dentro, es decir se debe hallar al descomponer la matriz
def matriz_vector_multiprocessing_p(cant_repeats, row, col, val, vector):
  #cant_repeats = [list(row).count(i) for i in range(int(max(row))+1)] #Crea un arreglo con la cantidad de datos diferentes a cero por fila
  cpus = os.cpu_count()
  chunk = 1
  if cpus > max(row)+1: cpus = max(row)+1 #Si hay más procesadores que procesos necesarios, entonces se recorta
  else: chunk = math.ceil((max(row)+1)/cpus) #Si hay menos, se divide en tantos procesos como procesadores y un chunk es un grupo de datos
  cpus = int(cpus)
  p = Pool(processes=cpus) #Inicializa el pool para multiprocessing
  args = zip((cant_repeats[i*chunk:(i+1)*chunk] for i in range(cpus)), #Manda las cantidades de datos diferentes a cero en las respectivas filas
             (len(cant_repeats[i*chunk:(i+1)*chunk]) for i in range(cpus)), #Manda el tamaño real de filas para evitar desbordamiento por el trabajo con chunks y saber como moverse
             (col[sum(cant_repeats[0:i*chunk]):sum(cant_repeats[0:i*chunk])+sum(cant_repeats[i*chunk:(i+1)*chunk])] for i in range(0, cpus)), #Manda el grupo de columnas respectivas dentro de las filas, ya que está ordenado por las filas y columnas
             (val[sum(cant_repeats[0:i*chunk]):sum(cant_repeats[0:i*chunk])+sum(cant_repeats[i*chunk:(i+1)*chunk])] for i in range(0, cpus)), #Misma lógica que para las columnas
             repeat(vector)) #Envía el vector igual siempre porque se necesita para multiplicar
  resultado = p.starmap(matriz_vector_multiprocessing_sub_p, args)
  return np.concatenate(resultado)

In [34]:
def matriz_vector_multiprocessing_sub_traditional_p(ini, fin, row, len_rep, col, val, vector):
  sol = np.zeros(len_rep).astype(np.float32) #fin-ini por la cantidad de filas que habrá
  for i in range(ini, fin-1): #Repite tantas filas hay por proceso
    if i-ini >= len_rep-1: break
    for j in range(int(row[i]), int(row[i+1])): #Itera cuantos datos diferentes a cero haya en la fila ahorrando iteraciones innecesarias
      sol[i-ini] += val[j] * vector[int(col[j])]
  if fin <= len(row)-1: max = int(row[fin]) #Compara para saber si se encuentra en el último puntero global o no, y así enviar su respecitvo fin de bucle
  else:
    max = len(col)
    fin = len(row)
  for i in range(int(row[fin-1]), max):
    sol[len_rep-1] += val[i] * vector[int(col[i])]
  return sol

def matriz_vector_multiprocessing_traditional_p(row, col, val, vector):
  cpus = os.cpu_count()
  chunk = 1
  if cpus > (len(row)): cpus = (len(row)) #Si hay más procesadores que procesos necesarios, entonces se recorta
  else: chunk = math.ceil((len(row))/cpus) #Si hay menos, se divide en tantos procesos como procesadores y un chunk es un grupo de datos
  cpus = int(cpus)
  p = Pool(processes=cpus) #Inicializa el pool para multiprocessing
  args = zip((i*chunk for i in range(cpus)), #Argumento del índice donde empieza el proceso
             ((i+1)*chunk for i in range(cpus)), #Argumento del índice donde acaba el proceso
             repeat(row),
             (len(row[i*chunk:(i+1)*chunk]) for i in range(cpus)),
             repeat(col),
             repeat(val),
             repeat(vector))
  resultado = p.starmap(matriz_vector_multiprocessing_sub_traditional_p, args)
  return np.concatenate(resultado)

In [31]:
m = 2**12
n = 2**12
sparse = sp.random(m, n, density=0.97, format='csr', dtype=np.float32).toarray()
sparse *= 10 #Consigue datos entre [0, 10[
vector = np.random.uniform(low=0, high=10, size=(n, )).astype(np.float32)

In [32]:
cant_rows, cant_cols, cant_per_row, row, col, val = compressed_sparse_row(sparse)
row_ptr_traditional, row_traditional, col_traditional, val_traditional = compressed_sparse_row_traditional(sparse)
#print(sparse)
#print(cant_per_row)
#print(cant_per_row, row, col, val, vector)
#n_sparse = decompressed_sparse_row(row, col, val, cant_rows, cant_cols)
#print(n_sparse)
#print(vector)

In [37]:
tic = time.perf_counter()
mult_sec = matriz_vector(cant_per_row, col, val, vector)
toc = time.perf_counter()
print(toc-tic)
#print(mult_sec)
tic = time.perf_counter()
mult_sec_trad = matriz_vector_traditional(row_ptr_traditional, col_traditional, val_traditional, vector)
toc = time.perf_counter()
print(toc-tic)
#print(mult_sec_trad)
tic = time.perf_counter()
mult_opt = matriz_vector_multiprocessing(cant_per_row, row, col, val, vector)
toc = time.perf_counter()
print(toc-tic)
#print(mult_opt)
tic = time.perf_counter()
mult_c = matriz_vector_c(cant_per_row, row, col, val, vector)
toc = time.perf_counter()
print(toc-tic)
#print(mult_c)
tic = time.perf_counter()
mult_p = matriz_vector_multiprocessing_p(cant_per_row, row, col, val, vector)
toc = time.perf_counter()
print(toc-tic)
#print(mult_p)
tic = time.perf_counter()
mult_simd = matriz_vector_traditional_simd(row_ptr_traditional, col_traditional, val_traditional, vector)
toc = time.perf_counter()
print(toc-tic)
#print(mult_simd)
tic = time.perf_counter()
mult_c_traditional = matriz_vector_traditional_c(row_ptr_traditional, col_traditional, val_traditional, vector)
toc = time.perf_counter()
print(toc-tic)
tic = time.perf_counter()
resultado = matriz_vector_multiprocessing_traditional_p(row_ptr_traditional, col_traditional, val_traditional, vector)
toc = time.perf_counter()
print(toc-tic)
#print(mult_c_traditional)

20.62668320900002
13.17312909799989
3.535371230999999
3.161788602000115
20.88356924300001
0.035435817000006864
0.07831837500020811
16.25348784200014
